# Offload and Idle Pull

This notebook demonstrates the workload behaviour for big core offload and idle pull

In [1]:
import logging
reload(logging)
logging.basicConfig(
    format='%(asctime)-9s %(levelname)-8s: %(message)s',
    datefmt='%I:%M:%S')

# Enable logging at INFO level
logging.getLogger().setLevel(logging.INFO)
# Comment the follwing line to disable devlib debugging statements
# logging.getLogger('ssh').setLevel(logging.DEBUG)

In [5]:
# Generate plots inline
%pylab inline

import json
import os
import subprocess

# Support to access the remote target
import devlib
from env import TestEnv

# Support for trace events analysis
from trace import Trace

# Support to configure and run RTApp based workloads
from wlgen import RTA

# Support for performance analysis of RTApp workloads
from perf_analysis import PerfAnalysis

# Suport for FTrace events parsing and visualization
import trappy

Populating the interactive namespace from numpy and matplotlib


# Test environment setup

In [3]:
# Setup a target configuration
my_target_conf = {
    
    "platform"    : 'android',
    "board"       : "hikey960",
    "device" : "0123456789ABCDEF",
    
    "ANDROID_HOME" : "/media/build/workspace/android-sdk",
    "rtapp-calib" : {"0": 302, "1": 302, "2": 304, "3": 304, "4": 136, "5": 137, "6": 136, "7": 136},

     "emeter" : {
        "instrument" : "acme",
        "conf" : {
            # Absolute path to the iio-capture binary on the host
            'iio-capture' : '/usr/bin/iio-capture',
            # Default host name of the BeagleBone Black
            'ip_address'     : 'baylibre-acme.local',
        },
        "channel_map" : {
            "Device0" : 0, # iio:device0
        }
    },

    # Define devlib module to load
    #"modules"     : [
    #    'bl',           # enable big.LITTLE support
    #    'cpufreq'       # enable CPUFreq support
    #],

    # Account to access the remote target
    "host"        : '10.169.36.25',
    "username"    : 'root',
    "password"    : '',



}

# Setup the required Test Environment supports
my_tests_conf = {
    
    # Binary tools required to run this experiment
    # These tools must be present in the tools/ folder for the architecture
    "tools"   : ['rt-app', 'taskset', 'trace-cmd'],
    
    # FTrace events end buffer configuration
    "ftrace"  : {
         "events" : [
            "cpu_frequency",
            "sched_load_avg_cpu",
            "sched_load_avg_task",
            "sched_switch",
            'dequeue_task_fair', 
            'enqueue_task_fair', 
            'set_next_entity',

                    
            "cpu_capacity",
            
            #/sys/kernel/debug/tracing/events/sched
            "sched_blocked_reason",
            "sched_boost_cpu",
            "sched_boost_task",
            "sched_contrib_scale_f",
            "sched_cpu_hotplug",
            "sched_energy_diff",
            "sched_kthread_stop",
            "sched_kthread_stop_ret",
            "sched_load_avg_cpu",
            "sched_load_avg_task",
            "sched_migrate_task",
            "sched_move_numa",
            "sched_pi_setprio",
            "sched_process_exec",
            "sched_process_exit",
            "sched_process_fork",
            "sched_process_free",
            "sched_process_wait",
            "sched_stat_blocked",
            "sched_stat_iowait",
            "sched_stat_runtime",
            "sched_stat_sleep",
            "sched_stat_wait",
            "sched_stick_numa",
            "sched_swap_numa",
            "sched_switch",
            "sched_tune_boostgroup_update",
            "sched_tune_config",
            "sched_tune_tasks_update",
            "sched_tune_filter",
            "sched_wait_task",
            "sched_wake_idle_without_ipi",
            "sched_wakeup",
            "sched_wakeup_new",
            
            
         ],
         "buffsize" : 10240
    },
    "results_dir" : "lisa_offload_and_idle_pull",
}

In [4]:
# Support to access the remote target
import devlib
from env import TestEnv

# Initialize a test environment using:
# the provided target configuration (my_target_conf)
# the provided test configuration   (my_test_conf)
te = TestEnv(target_conf=my_target_conf, test_conf=my_tests_conf)
target = te.target

05:46:47  INFO    : Using base path: /media/build/workspace/lisa-github
05:46:47  INFO    : Loading custom (inline) target configuration
05:46:47  INFO    : Loading custom (inline) test configuration
05:46:47  INFO    : External tools using:
05:46:47  INFO    :    ANDROID_HOME: /media/build/workspace/android-sdk
05:46:47  INFO    :    CATAPULT_HOME: /media/build/workspace/lisa-github/tools/catapult
05:46:47  INFO    : Devlib modules to load: ['bl', 'cpuidle', 'cpufreq']
05:46:47  INFO    : Connecting Android target [0123456789ABCDEF]
05:46:47  INFO    : Connection settings:
05:46:47  INFO    :    {'device': '0123456789ABCDEF'}
05:46:48  INFO    : Initializing target workdir:
05:46:48  INFO    :    /data/local/tmp/devlib-target
05:46:52  INFO    : Attempting to read energy model from target
05:46:57  INFO    : Topology:
05:46:57  INFO    :    [[0, 1, 2, 3], [4, 5, 6, 7]]
05:46:57  INFO    : Loading default EM:
05:46:57  INFO    :    /media/build/workspace/lisa-github/libs/utils/platform

# Workload configuration

In [6]:
# Support to configure and run RTApp based workloads
from wlgen import RTA, Periodic, Ramp

# Create a new RTApp workload generator using the calibration values
# reported by the TestEnv module
rtapp = RTA(target, 'simple', calibration=te.calibration())

# Configure this RTApp instance to:
rtapp.conf(
    # 1. generate a "profile based" set of tasks
    kind='profile',
    
    # 2. define the "profile" of each task
    params={
        
        "early_starter_1" : Periodic(
            period_ms=10,
            duty_cycle_pct=100,
            duration_s=5,
        ).get(),

        "early_starter_2" : Periodic(
            period_ms=10,
            duty_cycle_pct=100,
            duration_s=5,
        ).get(),

        
        "delay_starter_1" : Periodic(
            period_ms=10,
            duty_cycle_pct=100,
            duration_s=5,
            delay_s=1,
        ).get(),
        
        "delay_starter_2" : Periodic(
            period_ms=10,
            duty_cycle_pct=5,
            duration_s=10,
            delay_s=1,
        ).get(),

#        "delay_starter_3" : Periodic(
#            period_ms=10,
#            duty_cycle_pct=100,
#            duration_s=5,
#            delay_s=2,
#        ).get(),
#        
#        "delay_starter_4" : Periodic(
#            period_ms=10,
#            duty_cycle_pct=100,
#            duration_s=5,
#            delay_s=2,
#        ).get(),
    },
    
    # 4. use this folder for task logfiles
    run_dir=target.working_directory
    
);

05:47:43  INFO    : Setup new workload simple
05:47:43  INFO    : Workload duration defined by longest task
05:47:43  INFO    : Default policy: SCHED_OTHER
05:47:43  INFO    : ------------------------
05:47:43  INFO    : task [delay_starter_1], sched: using default policy
05:47:43  INFO    :  | start delay: 1.000000 [s]
05:47:43  INFO    :  | loops count: 1
05:47:43  INFO    :  + phase_000001: batch 5.000000 [s]
05:47:43  INFO    : ------------------------
05:47:43  INFO    : task [delay_starter_2], sched: using default policy
05:47:43  INFO    :  | start delay: 1.000000 [s]
05:47:43  INFO    :  | loops count: 1
05:47:43  INFO    : + phase_000001: duration 10.000000 [s] (1000 loops)
05:47:43  INFO    : |  period    10000 [us], duty_cycle   5 %
05:47:43  INFO    : |  run_time    500 [us], sleep_time   9500 [us]
05:47:43  INFO    : ------------------------
05:47:43  INFO    : task [early_starter_1], sched: using default policy
05:47:43  INFO    :  | loops count: 1
05:47:43  INFO    :  + 

# Workload execution

In [7]:
# Set Sched governor
#
# For Juno big/LITTLE CPUs
#
logging.info("Target ABI: %s, CPus: %s",
             target.abi,
             target.cpuinfo.cpu_names)

target.cpufreq.set_all_governors('sched')

logging.info("Target current governor: %s",
             target.read_value("/sys/devices/system/cpu/cpu0/cpufreq/scaling_governor")
            )
logging.info("Target big CPU max CPUfreq:%s\n\t\t    Target big CPU current CPUfreq: %s", 
             target.read_value("/sys/devices/system/cpu/cpu4/cpufreq/scaling_max_freq"),
             target.read_value("/sys/devices/system/cpu/cpu4/cpufreq/scaling_cur_freq")
             )

logging.info("Target LITTLE CPU max CPUfreq:%s \n\t\t    Target LITTLE CPU current CPUfreq: %s", 
             target.read_value("/sys/devices/system/cpu/cpu0/cpufreq/scaling_max_freq"),
             target.read_value("/sys/devices/system/cpu/cpu0/cpufreq/scaling_cur_freq")
             )

05:48:41  INFO    : Target ABI: arm64, CPus: ['A53', 'A53', 'A53', 'A53', 'A73', 'A73', 'A73', 'A73']
05:48:41  INFO    : Target current governor: sched
05:48:41  INFO    : Target big CPU max CPUfreq:2362000
		    Target big CPU current CPUfreq: 1421000
05:48:42  INFO    : Target LITTLE CPU max CPUfreq:1844000 
		    Target LITTLE CPU current CPUfreq: 1402000


In [8]:
logging.info('#### Setup FTrace')
te.ftrace.start()

logging.info('#### Start energy sampling')
te.emeter.reset()

logging.info('#### Start RTApp execution')
rtapp.run(out_dir=te.res_dir, cgroup="")

logging.info('#### Read energy consumption: %s/energy.json', te.res_dir)
nrg_report = te.emeter.report(out_dir=te.res_dir)

logging.info('#### Stop FTrace')
te.ftrace.stop()

trace_file = os.path.join(te.res_dir, 'trace.dat')
logging.info('#### Save FTrace: %s', trace_file)
te.ftrace.get_trace(trace_file)

logging.info('#### Save platform description: %s/platform.json', te.res_dir)
(plt, plt_file) = te.platform_dump(te.res_dir)

05:50:02  INFO    : #### Setup FTrace
05:50:05  INFO    : #### Start energy sampling


05:50:06  INFO    : #### Start RTApp execution
05:50:06  INFO    : Workload execution START:
05:50:06  INFO    :    /data/local/tmp/bin/rt-app /data/local/tmp/devlib-target/simple_00.json 2>&1
05:50:18  INFO    : #### Read energy consumption: /media/build/workspace/lisa-github/results/lisa_offload_and_idle_pull/energy.json
05:50:18  ERROR   : /usr/bin/iio-capture terminated for Device0 (iio:device0)
05:50:18  ERROR   : [ERROR: Unable to find host: Name or service not known
Unable to create IIO context
]
05:50:18  INFO    : #### Stop FTrace
05:50:18  INFO    : #### Save FTrace: /media/build/workspace/lisa-github/results/lisa_offload_and_idle_pull/trace.dat
05:50:21  INFO    : #### Save platform description: /media/build/workspace/lisa-github/results/lisa_offload_and_idle_pull/platform.json


# Collected results

In [9]:
# All data are produced in the output folder defined by the TestEnv module
logging.info('Content of the output folder %s', te.res_dir)
!ls -la {te.res_dir}

05:50:24  INFO    : Content of the output folder /media/build/workspace/lisa-github/results/lisa_offload_and_idle_pull


total 13528
drwxrwxr-x  2 steven steven     4096 Aug  9 17:50 .
drwxrwxr-x 40 steven steven     4096 Aug  9 17:46 ..
-rw-rw-r--  1 steven steven        2 Aug  9 17:50 energy.json
-rw-rw-r--  1 steven steven        2 Aug  9 17:50 energy_stats.json
-rw-rw-r--  1 steven steven      581 Aug  9 17:50 output.log
-rw-rw-r--  1 steven steven     1518 Aug  9 17:50 platform.json
-rw-r--r--  1 steven steven      284 Aug  9 17:50 rt-app-delay_starter_1-0.log
-rw-r--r--  1 steven steven   124160 Aug  9 17:50 rt-app-delay_starter_2-1.log
-rw-r--r--  1 steven steven      284 Aug  9 17:50 rt-app-early_starter_1-2.log
-rw-r--r--  1 steven steven      284 Aug  9 17:50 rt-app-early_starter_2-3.log
-rw-r--r--  1 steven steven     1396 Aug  9 17:50 simple_00.json
-rw-r--r--  1 steven steven 13684736 Aug  9 17:50 trace.dat


In [10]:
# Inspect the JSON file used to run the application
with open('{}/simple_00.json'.format(te.res_dir), 'r') as fh:
    rtapp_json = json.load(fh, )
logging.info('Generated RTApp JSON file:')
#print json.dumps(rtapp_json, indent=4, sort_keys=True)

05:50:27  INFO    : Generated RTApp JSON file:


In [12]:
# Dump the energy measured for the LITTLE and big clusters
logging.info('Energy: %s', nrg_report.report_file)
print json.dumps(nrg_report.channels, indent=4, sort_keys=True)

05:50:47  INFO    : Energy: /media/build/workspace/lisa-github/results/lisa_offload_and_idle_pull/energy.json


{}


In [13]:
# Dump the platform descriptor, which could be useful for further analysis
# of the generated results
logging.info('Platform description: %s', plt_file)
#print json.dumps(plt, indent=4, sort_keys=True)

05:50:50  INFO    : Platform description: /media/build/workspace/lisa-github/results/lisa_offload_and_idle_pull/platform.json


# Trace inspection

In [14]:
# Suport for FTrace events parsing and visualization
import trappy

# NOTE: The interactive trace visualization is available only if you run
#       the workload to generate a new trace-file
trappy.plotter.plot_trace(te.res_dir)

# New Task Behavior

In [15]:
events_to_parse = my_tests_conf['ftrace']['events']

trace = Trace(plt, te.res_dir, events_to_parse)

ftrace = trace.ftrace

trappy.plotter.plot_trace(ftrace, execnames=['early_starter_1', "early_starter_2", "delay_starter_1",
                                             "delay_starter_2", "delay_starter_3", "delay_starter_4",
                                            ])

/media/build/workspace/lisa-github/libs/trappy/trappy/ftrace.py:205: UserWarning: TRAPpy: Couldn't read <trappy.dynamic.set_next_entity object at 0x7f862dcd05d0> from cache, reading it from trace
  warnings.warn(warnstr)
/media/build/workspace/lisa-github/libs/trappy/trappy/ftrace.py:205: UserWarning: TRAPpy: Couldn't read <trappy.dynamic.cpu_frequency_devlib object at 0x7f862dcd0710> from cache, reading it from trace
  warnings.warn(warnstr)
/media/build/workspace/lisa-github/libs/trappy/trappy/ftrace.py:205: UserWarning: TRAPpy: Couldn't read <trappy.dynamic.sched_move_numa object at 0x7f862dcd0810> from cache, reading it from trace
  warnings.warn(warnstr)
/media/build/workspace/lisa-github/libs/trappy/trappy/ftrace.py:205: UserWarning: TRAPpy: Couldn't read <trappy.dynamic.sched_wait_task object at 0x7f862dcd0910> from cache, reading it from trace
  warnings.warn(warnstr)
/media/build/workspace/lisa-github/libs/trappy/trappy/ftrace.py:205: UserWarning: TRAPpy: Couldn't read <trappy

# Trace Overview

In [16]:
# NOTE: The interactive trace visualization is available only if you run
#       the workload to generate a new trace-file
trappy.plotter.plot_trace(ftrace)